In [2]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'
from tensorflow import keras
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import decode_predictions
from keras.models import Model, load_model
import keras.backend as K

import model as tcav_model
import tcav as tcav
import utils as utils
import activation_generator as act_gen
import tensorflow as tf
import utils_plot as utils_plot
import cav as cav

from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from numpy.linalg import norm

import pickle

from os import listdir
from os.path import isfile, join
import operator
from PIL import Image
from keras.preprocessing import image

import os
import math

import PIL.Image
from sklearn.metrics import pairwise
import matplotlib.pyplot as plt
from keras.applications.inception_v3 import preprocess_input

from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import train_test_split
from scipy.stats import multivariate_normal
from sklearn.mixture import GaussianMixture

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

import pandas as pd
from scipy import stats

import tensorflow



Using TensorFlow backend.


In [3]:
K.set_learning_phase(0)
model = load_model('v3_model.h5')

/Users/tyler/anaconda3/lib/python3.6/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [4]:
sess = K.get_session()

endpoints_v3 = dict(
    input=model.inputs[0].name,
    input_tensor=model.inputs[0],
    logit=model.outputs[0].name,
    prediction=model.outputs[0].name,
    prediction_tensor=model.outputs[0],)

tf.logging.set_verbosity(0)

working_dir = '/Users/tyler/Desktop/dissertation/programming/tcav_on_azure'

label_path = os.path.join(working_dir,'labels.txt')

mymodel = tcav_model.KerasModelWrapper(sess, 
        label_path, [299, 299, 3], endpoints_v3, 
        'InceptionV3_public', (-1, 1))

In [118]:
#df

In [119]:
columns = ['layer','concept_p','concept_n','acc_orig','acc_marg','acc_L1']
df = pd.DataFrame(columns = columns)

In [120]:
pickle.dump(df,open('df_acc1.pkl', 'wb'))

In [133]:
## CAV loop
source_dir = working_dir + '/concepts/'

target_class = 'zebra'
target_class_name = 'zebra'
#class_1 = 'grassland_sub_1'

#class_1_list = ['green_sub_1','green_sub_2','yellow_sub_1','yellow_sub_2']#,'blue_sub_1','blue_sub_2','orange_sub_1','orange_sub_1']
class_1_list = ['dotted_sub_1','sky_sub_1','ocean_sub_1']#,'blue_sub_1','blue_sub_2','orange_sub_1','orange_sub_1']

#class_1_list = ['random500_6','random500_7','random500_8','random500_9']
#class_1_list = ['sky_sub_1','ocean_sub_1','grassland_sub_1','road_sub_1']
#class_1_list = ['striped_sub_1','grid_sub_1','dotted_sub_1','banded_sub_1']
class_1_list = ['sky_sub_2']

hparams = tf.contrib.training.HParams(model_type='linear', alpha=.1)

bn_names = ['mixed0','mixed1','mixed2','mixed3','mixed4','mixed5','mixed6','mixed7','mixed8','mixed9']#,'mixed10']
#bn_names = ['mixed9']

df = pickle.load(open('df_acc1.pkl', 'rb'))

num_random = 5

for concept_p in class_1_list:
    for layer in  bn_names:
        for class_idx in range(num_random):
            concept_n = 'random500_' + str(class_idx)
            subset = df[(df['layer']==layer) & 
                    (df['concept_p']==concept_p) &
                    (df['concept_n']==concept_n)]
            if len(subset) == 0:
                print(f'running {layer}, {concept_p}, {concept_n}')
                
                acts_p,_ = get_acts_for_concept(concept_p,layer)
                acts_n,_ = get_acts_for_concept(concept_n,layer)
                #_,acts_class = get_acts_for_concept(target_class,layer)
                
                x = np.concatenate((acts_p,acts_n))
                y = np.concatenate((np.zeros(50),np.ones(50)))

                x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, stratify=y)

                cav_orig,lm= make_orig_cav_and_lm(x_train,y_train)
                preds = lm.predict(x_test)
                acc_orig = metrics.accuracy_score(y_test,preds)

                n_seeds = 50
                cav_2_seeds = np.empty((n_seeds,cav_orig.shape[0]))
                b = np.empty((n_seeds,1))

                for seed in range(n_seeds):
                    cav_2_seeds[seed],b[seed] = make_orig_cav_seed(x_train,y_train,seed)

                lm1 = linear_model.SGDClassifier(alpha=.1)
                lm1.coef_ = np.expand_dims(cav_2_seeds.mean(axis=0),axis=0)
                lm1.intercept_ = b.mean(axis=0)
                lm1.classes_ = np.array([0., 1.])
                preds = lm1.predict(x_test)
                acc_marg = metrics.accuracy_score(y_test,preds)

                #lm2 = linear_model.SGDClassifier(alpha=.1)

                #cav_L1,mu_p,mu_n = make_L1_cav_and_means(x_train,y_train)
                #b_ = -.5 * (np.dot(mu_p,mu_p.T) - np.dot(mu_n,mu_n.T))

                #lm2.coef_ = -np.expand_dims(cav_L1,axis=0)
                #lm2.intercept_ = b_
                #lm2.classes_ = np.array([0, 1.])
                #preds = lm2.predict(x_test)
                #acc_L1 = metrics.accuracy_score(y_test,preds)

                this_dict = {'layer':layer,
                             'concept_p':concept_p,
                             'concept_n':concept_n,
                             'acc_orig':acc_orig,
                             'acc_marg':acc_marg,}
                             #'acc_L1':acc_L1,}
                df = df.append([this_dict])
            else:
                print(f'already in df {layer}, {concept_p}, {concept_n}')
            
pickle.dump(df,open('df_acc1.pkl', 'wb'))
print('### did write ###')



running mixed0, sky_sub_2, random500_0
running mixed0, sky_sub_2, random500_1
running mixed0, sky_sub_2, random500_2
running mixed0, sky_sub_2, random500_3
running mixed0, sky_sub_2, random500_4
running mixed1, sky_sub_2, random500_0
running mixed1, sky_sub_2, random500_1
running mixed1, sky_sub_2, random500_2
running mixed1, sky_sub_2, random500_3
running mixed1, sky_sub_2, random500_4
running mixed2, sky_sub_2, random500_0
running mixed2, sky_sub_2, random500_1
running mixed2, sky_sub_2, random500_2
running mixed2, sky_sub_2, random500_3
running mixed2, sky_sub_2, random500_4
running mixed3, sky_sub_2, random500_0
running mixed3, sky_sub_2, random500_1
running mixed3, sky_sub_2, random500_2
running mixed3, sky_sub_2, random500_3
running mixed3, sky_sub_2, random500_4
running mixed4, sky_sub_2, random500_0
running mixed4, sky_sub_2, random500_1
running mixed4, sky_sub_2, random500_2
running mixed4, sky_sub_2, random500_3
running mixed4, sky_sub_2, random500_4
running mixed5, sky_sub_2

In [26]:
acts_p,_ = get_acts_for_concept(concept_p,layer)
acts_n,_ = get_acts_for_concept(concept_n,layer)
_,acts_class = get_acts_for_concept(target_class,layer)

In [107]:
x = np.concatenate((acts_p,acts_n))
y = np.concatenate((np.zeros(50),np.ones(50)))

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, stratify=y)

In [108]:
cav_orig,lm= make_orig_cav_and_lm(x_train,y_train)
preds = lm.predict(x_test)
acc_orig = metrics.accuracy_score(y_test,preds)
print(acc_orig)

n_seeds = 50
cav_2_seeds = np.empty((n_seeds,cav_orig.shape[0]))
b = np.empty((n_seeds,1))

for seed in range(n_seeds):
    cav_2_seeds[seed],b[seed] = make_orig_cav_seed(x_train,y_train,seed)

lm1 = linear_model.SGDClassifier(alpha=.1)
lm1.coef_ = np.expand_dims(cav_2_seeds.mean(axis=0),axis=0)
lm1.intercept_ = b.mean(axis=0)
lm1.classes_ = np.array([0., 1.])
preds = lm1.predict(x_test)
acc_marg = metrics.accuracy_score(y_test,preds)
print(acc_marg)

lm2 = linear_model.SGDClassifier(alpha=.1)

cav_L1,mu_p,mu_n = make_L1_cav_and_means(x_train,y_train)
b_ = -.5 * (np.dot(mu_p,mu_p.T) - np.dot(mu_n,mu_n.T))

lm2.coef_ = -np.expand_dims(cav_L1,axis=0)
lm2.intercept_ = b_
lm2.classes_ = np.array([0, 1.])
preds = lm2.predict(x_test)
acc_L1 = metrics.accuracy_score(y_test,preds)
print(acc_L1)

0.7272727272727273
0.9090909090909091


0.8181818181818182

In [54]:
## get activations for concept, layer
concept_dir = 'concepts/noise_white'
image_path = 'img1.jpg'
img = prep2(os.path.join(concept_dir,image_path))
this_img = np.expand_dims(img, axis=0)

def get_acts_for_concept(concept,layer):
    concept_dir = os.path.join(working_dir,'concepts/'+concept)
    image_list = files_from_dir_ext(concept_dir,'jp')
    image_list.sort()

    act_path = os.path.join(working_dir,'final_acts/' + concept + '-' + layer + '.pkl')

    n = size_dict[layer]
    nn = size_dict_orig[layer]

    try:
        this_dict = pickle.load(open(act_path, 'rb'))
    except:
        this_dict = {}
        
    #print(nn)
    
    acts_ran = np.zeros((len(image_list),n))
    orig = np.zeros((len(image_list),nn[1],nn[2],nn[3]))
    
    for idx,image_path in enumerate(image_list):
        if image_path not in this_dict:
            img = prep2(os.path.join(concept_dir,image_path))
            this_img = np.expand_dims(img, axis=0)
            acts_orig = get_acts_for_layer_new(layer,this_img)
            acts_ran[idx] = acts_orig.reshape(-1)
            orig[idx] = acts_orig
            this_dict[image_path] = (acts_orig.reshape(-1),acts_orig)
        else:
            acts_ran[idx],orig[idx] = this_dict[image_path]
            #print('acts already exist')

    pickle.dump(this_dict,open(act_path, 'wb'))
    
    return acts_ran,orig

def make_orig_cav(x_train,y_train):
    lm = linear_model.SGDClassifier(alpha=.1)
    lm.fit(x_train, y_train)
    this_cav = -lm.coef_[0]
    return this_cav

def make_orig_cav_and_lm(x_train,y_train):
    lm = linear_model.SGDClassifier(alpha=.1)
    lm.fit(x_train, y_train)
    this_cav = -lm.coef_[0]
    return this_cav,lm

def make_orig_cav_seed(x_train,y_train,seed):
    lm = linear_model.SGDClassifier(alpha=.1,random_state=seed)
    lm.fit(x_train, y_train)
    return lm.coef_,lm.intercept_

def make_L1_cav(x_train,y_train):
    x_train_p_list,x_train_n_list =[],[]
    for idx,a in enumerate(x_train):
        if y_train[idx] == 0:
            x_train_p_list.append(a)
        else:
            x_train_n_list.append(a)

    x_train_p, x_train_n = np.array(x_train_p_list),np.array(x_train_n_list)
    L1_cav = get_L1(x_train_p) - get_L1(x_train_n)
    return L1_cav

def make_centroid_cav(x_train,y_train):
    x_train_p_list,x_train_n_list =[],[]
    for idx,a in enumerate(x_train):
        if y_train[idx] == 0:
            x_train_p_list.append(a)
        else:
            x_train_n_list.append(a)

    x_train_p, x_train_n = np.array(x_train_p_list),np.array(x_train_n_list)
    centroid_cav = x_train_p.mean(axis=1) - x_train_n.mean(axis=1)
    return centroid_cav

def make_L1_cav_and_means(x_train,y_train):
    x_train_p_list,x_train_n_list =[],[]
    for idx,a in enumerate(x_train):
        if y_train[idx] == 0:
            x_train_p_list.append(a)
        else:
            x_train_n_list.append(a)

    x_train_p, x_train_n = np.array(x_train_p_list),np.array(x_train_n_list)
    mu_p = get_L1(x_train_p)
    mu_n = get_L1(x_train_n)
    L1_cav = mu_p - mu_n
    return L1_cav,mu_p,mu_n

def get_L1(act_for_L1):
    return np.linalg.norm(act_for_L1.T,1,axis = 1) / act_for_L1.shape[1]

def files_from_dir_ext(a_dir,ext):
    onlyfiles = [f for f in os.listdir(a_dir) if os.path.isfile(os.path.join(a_dir, f))]
    this_ext = [e for e in onlyfiles if ext in e.lower()]
    return this_ext

acts_mixed0_f = K.function([model.input],[layer_dict['mixed0'].output])
acts_mixed1_f = K.function([model.input],[layer_dict['mixed1'].output])
acts_mixed2_f = K.function([model.input],[layer_dict['mixed2'].output])
acts_mixed3_f = K.function([model.input],[layer_dict['mixed3'].output])
acts_mixed4_f = K.function([model.input],[layer_dict['mixed4'].output])
acts_mixed5_f = K.function([model.input],[layer_dict['mixed5'].output])
acts_mixed6_f = K.function([model.input],[layer_dict['mixed6'].output])
acts_mixed7_f = K.function([model.input],[layer_dict['mixed7'].output])
acts_mixed8_f = K.function([model.input],[layer_dict['mixed8'].output])
acts_mixed9_f = K.function([model.input],[layer_dict['mixed9'].output])
acts_mixed10_f = K.function([model.input],[layer_dict['mixed10'].output])

    
def get_acts_for_layer_new(layer_name,input_img):
    acts = None
    if layer_name=='mixed0':
        acts = acts_mixed0_f([input_img])[0]
    if layer_name=='mixed1':
        acts = acts_mixed1_f([input_img])[0]
    if layer_name=='mixed2':
        acts = acts_mixed2_f([input_img])[0]
    if layer_name=='mixed3':
        acts = acts_mixed3_f([input_img])[0]
    if layer_name=='mixed4':
        acts = acts_mixed4_f([input_img])[0]
    if layer_name=='mixed5':
        acts = acts_mixed5_f([input_img])[0]
    if layer_name=='mixed6':
        acts = acts_mixed6_f([input_img])[0]
    if layer_name=='mixed7':
        acts = acts_mixed7_f([input_img])[0]
    if layer_name=='mixed8':
        acts = acts_mixed8_f([input_img])[0]
    if layer_name=='mixed9':
        acts = acts_mixed9_f([input_img])[0]
    if layer_name=='mixed10':
        acts = acts_mixed10_f([input_img])[0]
    
    return acts


bn_names = ['mixed0','mixed1','mixed2','mixed3','mixed4','mixed5','mixed6','mixed7','mixed8','mixed9','mixed10']

size_dict = {}
for bn in bn_names:
    acts_orig = get_acts_for_layer_new(bn,this_img)
    size_dict[bn] = acts_orig.reshape(-1).shape[0]
    
size_dict_orig = {}
for bn in bn_names:
    acts_orig = get_acts_for_layer_new(bn,this_img)
    size_dict_orig[bn] = acts_orig.shape


In [24]:
layer_dict = dict([(layer.name, layer) for layer in model.layers])

In [21]:
def prep2(filename):
    shape=(299, 299)
    img = np.array(PIL.Image.open(open(filename, 'rb')).convert('RGB').resize(shape, PIL.Image.BILINEAR))
    # Normalize pixel values to between 0 and 1.
    img = np.float32(img) / 255.0
    if not (len(img.shape) == 3 and img.shape[2] == 3):
        return None
    else:
        return img